In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('cleveland297.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'ca', 'thal', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,57,0,4,140,241,0,0,123,1,0.2,2,0,7,1
293,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
294,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
295,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3


In [3]:
### 1 = male, 0 = female
df.isnull().sum()

df['target'] = df.target.map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})
df['sex'] = df.sex.map({0: 'female', 1: 'male'})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())
df['sex'] = df.sex.map({'female': 0, 'male': 1})

In [4]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [5]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [6]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [7]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [16]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [17]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
6/6 [==============================] - 2s 71ms/step - loss: 0.6956 - accuracy: 0.6105 - val_loss: 0.6655 - val_accuracy: 0.6000
Epoch 2/100
6/6 [==============================] - 0s 17ms/step - loss: 0.5503 - accuracy: 0.7453 - val_loss: 0.6424 - val_accuracy: 0.7667
Epoch 3/100
6/6 [==============================] - 0s 16ms/step - loss: 0.5611 - accuracy: 0.7303 - val_loss: 0.6256 - val_accuracy: 0.8000
Epoch 4/100
6/6 [==============================] - 0s 36ms/step - loss: 0.4790 - accuracy: 0.7753 - val_loss: 0.6122 - val_accuracy: 0.7333
Epoch 5/100
6/6 [==============================] - 0s 37ms/step - loss: 0.4364 - accuracy: 0.7903 - val_loss: 0.6094 - val_accuracy: 0.7333
Epoch 6/100
6/6 [==============================] - 0s 36ms/step - loss: 0.4611 - accuracy: 0.8052 - val_loss: 0.6036 - val_accuracy: 0.7667
Epoch 7/100
6/6 [==============================] - 0s 35ms/step - loss: 0.4340 - accuracy: 0.8090 - val_loss: 0.5931 - val_accuracy: 0.7333
Epoch 8/100
6/6 [===

6/6 [==============================] - 0s 16ms/step - loss: 0.0901 - accuracy: 0.9700 - val_loss: 0.4327 - val_accuracy: 0.8000
Epoch 60/100
6/6 [==============================] - 0s 34ms/step - loss: 0.1303 - accuracy: 0.9513 - val_loss: 0.4082 - val_accuracy: 0.8333
Epoch 61/100
6/6 [==============================] - 0s 35ms/step - loss: 0.0768 - accuracy: 0.9850 - val_loss: 0.3864 - val_accuracy: 0.8333
Epoch 62/100
6/6 [==============================] - 0s 14ms/step - loss: 0.1371 - accuracy: 0.9326 - val_loss: 0.3939 - val_accuracy: 0.8000
Epoch 63/100
6/6 [==============================] - 0s 16ms/step - loss: 0.0873 - accuracy: 0.9700 - val_loss: 0.4226 - val_accuracy: 0.7667
Epoch 64/100
6/6 [==============================] - 0s 14ms/step - loss: 0.0807 - accuracy: 0.9775 - val_loss: 0.4101 - val_accuracy: 0.8333
Epoch 65/100
6/6 [==============================] - 0s 16ms/step - loss: 0.0987 - accuracy: 0.9663 - val_loss: 0.4454 - val_accuracy: 0.8333
Epoch 66/100
6/6 [========

In [18]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


1/1 [==============================] - 0s 32ms/step - loss: 0.3736 - accuracy: 0.8667
Test Loss: 0.3736
Test Accuracy: 0.8667


In [19]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


1/1 [==============================] - 0s 161ms/step
Confusion Matrix:
[[17  2]
 [ 2  9]]

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89        19
           1       0.82      0.82      0.82        11

    accuracy                           0.87        30
   macro avg       0.86      0.86      0.86        30
weighted avg       0.87      0.87      0.87        30



In [12]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [13]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [14]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
6/6 [==============================] - 14s 532ms/step - loss: 0.6746 - accuracy: 0.6479 - val_loss: 0.6904 - val_accuracy: 0.5667
Epoch 2/100
6/6 [==============================] - 0s 62ms/step - loss: 0.6086 - accuracy: 0.7228 - val_loss: 0.6730 - val_accuracy: 0.7000
Epoch 3/100
6/6 [==============================] - 0s 64ms/step - loss: 0.5756 - accuracy: 0.7228 - val_loss: 0.6380 - val_accuracy: 0.7333
Epoch 4/100
6/6 [==============================] - 0s 64ms/step - loss: 0.5564 - accuracy: 0.7303 - val_loss: 0.6245 - val_accuracy: 0.7000
Epoch 5/100
6/6 [==============================] - 0s 42ms/step - loss: 0.5599 - accuracy: 0.7303 - val_loss: 0.6307 - val_accuracy: 0.6333
Epoch 6/100
6/6 [==============================] - 0s 44ms/step - loss: 0.5394 - accuracy: 0.7341 - val_loss: 0.6368 - val_accuracy: 0.6333
Epoch 7/100
6/6 [==============================] - 0s 40ms/step - loss: 0.5222 - accuracy: 0.7341 - val_loss: 0.6397 - val_accuracy: 0.6333
Epoch 8/100
6/6 [=

In [15]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


1/1 [==============================] - 0s 40ms/step - loss: 0.6245 - accuracy: 0.7000
Test Loss: 0.6245
Test Accuracy: 0.7000


In [20]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


1/1 [==============================] - 2s 2s/step
Confusion Matrix:
[[19  0]
 [ 9  2]]

Classification Report:
              precision    recall  f1-score   support

           0       0.68      1.00      0.81        19
           1       1.00      0.18      0.31        11

    accuracy                           0.70        30
   macro avg       0.84      0.59      0.56        30
weighted avg       0.80      0.70      0.62        30



In [21]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [28]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [29]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
6/6 [==============================] - 19s 619ms/step - loss: 0.6282 - accuracy: 0.7191 - val_loss: 0.6910 - val_accuracy: 0.3667
Epoch 2/100
6/6 [==============================] - 0s 74ms/step - loss: 0.4990 - accuracy: 0.7940 - val_loss: 0.6611 - val_accuracy: 0.9333
Epoch 3/100
6/6 [==============================] - 0s 74ms/step - loss: 0.4493 - accuracy: 0.7978 - val_loss: 0.6560 - val_accuracy: 0.7000
Epoch 4/100
6/6 [==============================] - 0s 74ms/step - loss: 0.4321 - accuracy: 0.7828 - val_loss: 0.6331 - val_accuracy: 0.7333
Epoch 5/100
6/6 [==============================] - 0s 76ms/step - loss: 0.4172 - accuracy: 0.8240 - val_loss: 0.6212 - val_accuracy: 0.7000
Epoch 6/100
6/6 [==============================] - 0s 74ms/step - loss: 0.4343 - accuracy: 0.7940 - val_loss: 0.5213 - val_accuracy: 0.7000
Epoch 7/100
6/6 [==============================] - 0s 72ms/step - loss: 0.4173 - accuracy: 0.8202 - val_loss: 0.5322 - val_accuracy: 0.6667
Epoch 8/100
6/6 [=

In [33]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


1/1 [==============================] - 0s 48ms/step - loss: 0.2894 - accuracy: 0.9000
Test Loss: 0.2894
Test Accuracy: 0.9000


In [34]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


1/1 [==============================] - 0s 40ms/step
Confusion Matrix:
[[19  0]
 [ 3  8]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.93        19
           1       1.00      0.73      0.84        11

    accuracy                           0.90        30
   macro avg       0.93      0.86      0.88        30
weighted avg       0.91      0.90      0.90        30



In [35]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



1/1 [==============================] - 0s 48ms/step


In [36]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [37]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8667


In [38]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[19  0]
 [ 4  7]]
Classification Report:
              precision    recall  f1-score   support

           0       0.83      1.00      0.90        19
           1       1.00      0.64      0.78        11

    accuracy                           0.87        30
   macro avg       0.91      0.82      0.84        30
weighted avg       0.89      0.87      0.86        30

Ensemble Accuracy: 0.8667


In [39]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [40]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [41]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [42]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [43]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [44]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8667


In [45]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[17  2]
 [ 2  9]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89        19
           1       0.82      0.82      0.82        11

    accuracy                           0.87        30
   macro avg       0.86      0.86      0.86        30
weighted avg       0.87      0.87      0.87        30

Ensemble Accuracy: 0.8667


In [47]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [48]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.9000
Confusion Matrix:
[[18  1]
 [ 2  9]]
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92        19
           1       0.90      0.82      0.86        11

    accuracy                           0.90        30
   macro avg       0.90      0.88      0.89        30
weighted avg       0.90      0.90      0.90        30

Ensemble Accuracy: 0.9000
